# Klasyfikacja obrazów za pomocą CNN i zbioru danych Imagenette

&#x20;

## Wprowadzenie

Konwolucyjne sieci neuronowe (ang. *Convolutional Neural Networks*, **CNN**) to obecnie standardowa technika w zadaniach wizji komputerowej opartych na uczeniu głębokim. CNN wykorzystują warstwy splotowe (konwolucyjne), które automatycznie uczą się wychwytywać istotne cechy z obrazów (np. krawędzie, wzory, kształty) poprzez **filtry konwolucyjne**. Dzięki operacji **poolingu** (najczęściej *maksymalnego poolingowania*), sieć redukuje wymiarowość danych, zachowując najważniejsze informacje. Na końcu sieć zwykle posiada jedną lub więcej **warstw gęstych** (*fully connected*), które na podstawie wyekstrahowanych cech dokonują klasyfikacji do określonych klas. Taka architektura sprawia, że CNN świetnie radzą sobie z rozpoznawaniem obrazów, przewyższając tradycyjne metody opierające się na ręcznie tworzonych cechach.

**Imagenette** to niewielki zbiór danych obrazowych przygotowany przez zespół [fast.ai](http://fast.ai) jako uproszczona wersja słynnego zestawu **ImageNet**. Zawiera on około 13 tysięcy obrazów należących do 10 różnych, łatwo rozróżnialnych klas pochodzących z oryginalnego ImageNetu. Dzięki ograniczeniu liczby klas i przykładów, Imagenette umożliwia szybkie eksperymenty z modelami rozpoznawania obrazów, zanim zastosuje się je na pełnym, znacznie większym zbiorze ImageNet. Wśród kategorii obrazów w Imagenette znajdują się między innymi:

![](https://www.researchgate.net/publication/368652757/figure/fig2/AS:11431281121137743@1676862920482/Example-images-of-the-Imagenette-dataset-with-their-labels.ppm)



W kolejnych sekcjach przejdziemy przez następujące kroki:

1. Pobranie i przygotowanie zbioru danych **Imagenette**.
2. Wczytanie danych obrazowych z dysku przy użyciu generatorów Keras (`ImageDataGenerator`).
3. Zdefiniowanie architektury konwolucyjnej sieci neuronowej od podstaw.
4. Kompilacja i trenowanie modelu .
5. Ewaluacja modelu na danych walidacyjnych i ocena uzyskanej dokładności.


## Pobieranie i przygotowanie danych

Zacznijmy od pobrania zbioru Imagenette. Dane są dostępne publicznie – zbiór w wersji **160 px** (najkrótszy bok obrazów zeskalowany do 160 pikseli) można pobrać jako spakowany plik `.tgz` z zasobów [fast.ai](http://fast.ai). W poniższej komórce użyjemy polecenia `wget`, aby pobrać ten plik, a następnie `tar`, aby go rozpakować do katalogu lokalnego. Komendy `!wget` i `!tar` zostaną wykonane w środowisku notebooka jako polecenia systemowe (poprzedzone wykrzyknikiem).

In [ ]:
# Pobranie pliku z danymi Imagenette (wersja 160px)
!wget -q --show-progress https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-160.tgz

# Rozpakowanie archiwum .tgz do bieżącego katalogu
!tar -xzf imagenette2-160.tgz

# Usunięcie pobranego archiwum (opcjonalnie, aby zaoszczędzić miejsce)
!rm -f imagenette2-160.tgz

# Sprawdzenie zawartości rozpakowanego katalogu
!find imagenette2-160 -maxdepth 2 -type d




*Powyżej:* Najpierw pobieramy plik `imagenette2-160.tgz` (ok. 155 MB) poleceniem `wget`. Następnie rozpakowujemy go poleceniem `tar -xzf`. Po rozpakowaniu powinniśmy otrzymać katalog `imagenette2-160`, zawierający dwa podkatalogi: `train` (obrazy treningowe) oraz `val` (obrazy walidacyjne), a także plik `noisy_imagenette.csv` z informacjami o etykietach (z którego tutaj nie będziemy korzystać). Na końcu, dla porządku, usuwamy zbędne już archiwum oraz wypisujemy strukturę katalogów (do dwóch poziomów w głąb) aby zweryfikować, że dane zostały pobrane poprawnie.



Po wykonaniu powyższych poleceń, struktura katalogów powinna wyglądać następująco:

```

imagenette2-160/
├── train/
│   ├── n01440764/   (obrazy klasy "tench")
│   ├── n02102040/   (obrazy klasy "English springer")
│   ├── n02979186/   (obrazy klasy "cassette player")
│   ├── n03000684/   (obrazy klasy "chain saw")
│   ├── n03028079/   (obrazy klasy "church")
│   ├── n03394916/   (obrazy klasy "French horn")
│   ├── n03417042/   (obrazy klasy "garbage truck")
│   ├── n03425413/   (obrazy klasy "gas pump")
│   ├── n03445777/   (obrazy klasy "golf ball")
│   └── n03888257/   (obrazy klasy "parachute")
└── val/
    ├── n01440764/   (obrazy klasy "tench")
    ├── n02102040/   (obrazy klasy "English springer")
    ├── n02979186/   (obrazy klasy "cassette player")
    ├── n03000684/   (obrazy klasy "chain saw")
    ├── n03028079/   (obrazy klasy "church")
    ├── n03394916/   (obrazy klasy "French horn")
    ├── n03417042/   (obrazy klasy "garbage truck")
    ├── n03425413/   (obrazy klasy "gas pump")
    ├── n03445777/   (obrazy klasy "golf ball")
    └── n03888257/   (obrazy klasy "parachute")


```



Widzimy, że w katalogach `train` i `val` znajdują się podfoldery nazwane kodami WordNet (np. `n01440764`), odpowiadającymi poszczególnym klasom obrazów. Katalog `train` zawiera obrazy treningowe (około 70% danych), a `val` obrazy walidacyjne (około 30% danych) zgodnie z przygotowanym podziałem. Teraz, gdy dane są już na dysku, możemy przejść do ich wczytania i przygotowania do treningu sieci neuronowej.



## Przygotowanie danych z użyciem `ImageDataGenerator`

Do wczytywania obrazów skorzystamy z klasy **ImageDataGenerator** dostarczanej przez API Keras (część biblioteki TensorFlow). `ImageDataGenerator` umożliwia **wczytywanie obrazów bezpośrednio z folderów** oraz opcjonalne zastosowanie przekształceń zwiększających różnorodność danych treningowych (*augmentacja danych*), takich jak odbicia lustrzane, rotacje, zmiany jasności itp. W naszym przypadku zastosujemy podstawowe przekształcenia, aby model uczył się większej odporności na zmienność danych.

Najpierw zaimportujemy potrzebne moduły biblioteki TensorFlow i Keras, a następnie utworzymy dwa generatory:

* **train\_datagen** – generator dla danych treningowych, z augmentacją (np. losowe odbicie w poziomie) i normalizacją pikseli.
* **valid\_datagen** – generator dla danych walidacyjnych, bez augmentacji (służy jedynie do oceny modelu), z samą normalizacją pikseli.

Normalizacja polega tu na przemnożeniu wartości pikseli przez współczynnik `1/255`, tak aby zakres wartości (0-255 dla obrazów RGB) został przeskalowany do przedziału \[0,1]. Jest to standardowa operacja ułatwiająca trenowanie sieci neuronowej (dzięki temu wszystkie cechy wejściowe mają porównywalny zakres).

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Ustawienie podstawowych parametrów
IMG_WIDTH = 160   # docelowa szerokość obrazów (piksele)
IMG_HEIGHT = 160  # docelowa wysokość obrazów (piksele)
BATCH_SIZE = 32   # wielkość paczki (batch) wykorzystywanej podczas treningu



Powyżej zdefiniowaliśmy rozmiar obrazów (`160x160` pikseli) oraz wielkość batcha (32). Ponieważ korzystamy z wersji datasetu, gdzie najkrótszy bok obrazów ma 160px, przyjmiemy jednorodny rozmiar 160x160 dla wszystkich wejść sieci (obrazy zostaną automatycznie przeskalowane przez generator, jeśli mają inny rozmiar). Batch size określa, ile próbek będzie przetwarzanych jednocześnie przed zaktualizowaniem wag modelu podczas trenowania.

Teraz utworzymy obiekty `ImageDataGenerator` dla zbioru treningowego i walidacyjnego:

In [ ]:
# Tworzenie generatora dla danych treningowych z augmentacją
train_datagen = ImageDataGenerator(rescale=1.0/255.0,   # normalizacja pikseli
                                   horizontal_flip=True,  # losowe odbicie obrazu w poziomie
                                   rotation_range=20,     # losowa rotacja obrazów o maks. 20 stopni
                                   zoom_range=0.2,        # losowe przybliżenie obrazu
                                   width_shift_range=0.1, # losowe przesunięcie w poziomie o ±10% szerokości
                                   height_shift_range=0.1)# losowe przesunięcie w pionie o ±10% wysokości

# Generator dla danych walidacyjnych (tylko normalizacja, bez augmentacji)
valid_datagen = ImageDataGenerator(rescale=1.0/255.0)




W powyższym kodzie:

* `rescale=1.0/255.0` – przeskalowuje wartości pikseli.
* `horizontal_flip=True` – losowo odwraca część obrazów w poziomie.
* `rotation_range=20` – losowo obraca obrazy w zakresie ±20 stopni.
* `zoom_range=0.2` – losowo przybliża (zoom) obrazy o maksymalnie 20%.
* `width_shift_range` i `height_shift_range` – losowo przesuwa obrazy w poziomie i pionie (do 10% wymiaru).



Te operacje augmentacji będą **losowo** stosowane do obrazów treningowych przy każdym kroku epoki, co efektywnie zwiększa różnorodność danych treningowych i może poprawić uogólnianie się modelu. W zbiorze walidacyjnym natomiast nie chcemy dokonywać żadnych losowych zmian (walidacja powinna odbywać się na stałych danych), dlatego `valid_datagen` zawiera tylko normalizację.

Następnie za pomocą metody `flow_from_directory` utworzymy generatory danych obrazowych, które będą bezpośrednio dostarczać obrazy z odpowiednich folderów:


In [ ]:
# Ścieżki do katalogów z danymi
train_dir = "imagenette2-160/train"
valid_dir = "imagenette2-160/val"

# Tworzenie iteratora/generatora dla danych treningowych
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_WIDTH, IMG_HEIGHT),  # dopasowanie obrazów do rozmiaru 160x160
    batch_size=BATCH_SIZE,
    class_mode='categorical'  # typ etykiet: kategorie (one-hot vektory dla klasyfikacji wieloklasowej)
)

# Tworzenie iteratora/generatora dla danych walidacyjnych
valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)



Po wykonaniu powyższych poleceń, `flow_from_directory` przeskanuje foldery i wyświetli informacje, ile obrazów znalazł oraz ile klas zidentyfikował. Generatory przypiszą automatycznie etykiety numeryczne dla poszczególnych podfolderów (np. 0 dla `n01440764`, 1 dla `n02102040`, itd.), a dzięki `class_mode='categorical'` etykiety te będą zwracane w postaci wektorów one-hot o długości 10.

Możemy opcjonalnie sprawdzić, jakie klasy zostały rozpoznane przez generator:


In [ ]:
print("Mapowanie klas:", train_generator.class_indices)




## Budowa modelu CNN od podstaw

Zbudujemy konwolucyjną sieć neuronową korzystając z interfejsu **Keras** (API wysokiego poziomu dostępnego w TensorFlow). Użyjemy najprostszej formy definiowania modelu – sekwencyjnie (*Sequential*), ponieważ nasza architektura będzie stanowić liniowy stos warstw (od wejścia do wyjścia każda warstwa jest podłączona do następnej).

Architektura naszej sieci będzie zawierać następujące elementy:

* **Warstwy konwolucyjne (Conv2D)** – będą filtrować obrazy przy pomocy zestawu learnowalnych filtrów 2D, wykrywając lokalne wzorce. Użyjemy niewielkich filtrów (np. 3x3) i funkcji aktywacji **ReLU** (*Rectified Linear Unit*) po każdej konwolucji, co nadaje nieliniowość modelowi.
* **Warstwy poolingujące (MaxPooling2D)** – będą zmniejszać rozmiary map cech, agregując informacje (weźmiemy maksymalną wartość z okna 2x2). Pooling co pewien czas redukuje rozdzielczość reprezentacji, zmniejszając liczbę parametrów i uwzględniając pewną odporność na przesunięcia obrazu.
* **Warstwy gęste (Dense)** – na końcu sieci zastosujemy kilka neuronów w pełni połączonych. Szczególnie ostatnia warstwa będzie miała 10 neuronów (po jednym na każdą klasę) z aktywacją **softmax**, która przekształca wyjścia w prawdopodobieństwa klas.
* **Dropout** – dodamy warstwę *dropout* pomiędzy warstwami gęstymi, by przeciwdziałać overfittingowi. Warstwa ta losowo wyłącza pewien procent neuronów podczas treningu (np. 50%), co zmusza sieć do bardziej uogólnionego uczenia się cech.

In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Inicjalizacja modelu sequential
model = Sequential()

# 1. Warstwa konwolucyjna + aktywacja ReLU
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu',
                 input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)))
#    - 32 filtrów konwolucyjnych rozmiaru 3x3 pixeli
#    - funkcja aktywacji ReLU (Rectified Linear Unit)
#    - input_shape = (160, 160, 3) określa wymiar wejścia (kolorowe obrazy 160x160)

# 2. Druga warstwa konwolucyjna + ReLU
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
#    - kolejna warstwa konwolucyjna również z 32 filtrami 3x3

# 3. Warstwa pooling (maksymalne poolingowanie 2x2)
model.add(MaxPooling2D(pool_size=(2,2)))
#    - zmniejsza wymiary map cech o połowę (160x160 -> 80x80)
#    - bierze maksymalną wartość z każdej siatki 2x2

# 4. Warstwa dropout
model.add(Dropout(0.25))
#    - wyłącza losowo 25% neuronów tej warstwy podczas treningu, co pomaga zapobiegać przeuczeniu

# 5. Trzecia warstwa konwolucyjna + ReLU (z większą liczbą filtrów)
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
#    - 64 filtry 3x3, szuka bardziej złożonych cech w zredukowanych mapach cech

# 6. Czwarta warstwa konwolucyjna + ReLU
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))

# 7. Druga warstwa pooling 2x2
model.add(MaxPooling2D(pool_size=(2,2)))
#    - ponownie zmniejszamy wymiary (80x80 -> 40x40)

# 8. Warstwa dropout
model.add(Dropout(0.25))
#    - znów losowo wyłączamy 25% neuronów na tej warstwie podczas treningu

# 9. Spłaszczenie tensora (Flatten)
model.add(Flatten())
#    - przekształca 3-wymiarowe mapy cech (40x40 z 64 filtrami) na wektor 1D
#      celem podłączenia do warstwy gęstej

# 10. Warstwa gęsta (fully connected) + ReLU
model.add(Dense(units=128, activation='relu'))
#     - 128 neuronów w pełni połączonych z poprzednią warstwą
#     - ReLU jako funkcja aktywacji

# 11. Warstwa dropout (50%)
model.add(Dropout(0.5))
#     - wyłączamy losowo 50% neuronów warstwy Dense podczas treningu, żeby zredukować overfitting

# 12. Warstwa wyjściowa (Dense z softmax)
model.add(Dense(units=10, activation='softmax'))
#     - 10 neuronów odpowiadających 10 klasom; softmax zamienia wyjścia na prawdopodobieństwa klasy




Kilka uwag do powyższej architektury:

* Pierwsza warstwa `Conv2D` określa `input_shape=(160,160,3)`. Kolejne warstwy Keras potrafią już same wywnioskować kształty na podstawie poprzednich, więc nie wymagają jawnego podawania rozmiaru wejścia.
* Utrzymujemy względnie małą liczbę filtrów (32, potem 64) oraz dość agresywnie zmniejszamy wymiary za pomocą pooling (dwukrotnie). Dzięki temu liczba parametrów nie rośnie nadmiernie szybko. W ostatniej części sieci (po spłaszczeniu) mamy warstwę Dense 128 neuronów – to stosunkowo niewiele, co również ma ograniczyć złożoność modelu.
* Warstwy *Dropout* z ustawieniami 0.25 i 0.5 oznaczają, że podczas każdego kroku uczenia, odpowiednio 1/4 lub 1/2 neuronów z tych warstw jest pomijana. To pomaga uniknąć sytuacji, w której model zbytnio dostosowuje się do danych treningowych kosztem zdolności generalizacji (przeciwdziałanie przeuczeniu).

In [ ]:


Po zdefiniowaniu modelu warto sprawdzić podsumowanie jego architektury, co możemy zrobić za pomocą metody `model.summary()`:

In [ ]:

# Wyświetlenie podsumowania architektury modelu
model.summary()



Ta komenda wyświetli tabelaryczne zestawienie warstw modelu, w tym wymiary wyjść każdej warstwy oraz łączną liczbę parametrów do wytrenowania. Powinniśmy zobaczyć kolejno warstwy konwolucyjne, pooling, dropout itp., aż do warstwy wyjściowej, a na końcu sumaryczną liczbę parametrów (kilkaset tysięcy, w zależności od dokładnej architektury).

## Kompilacja i trenowanie modelu

Mając zdefiniowany model, musimy go **skompilować**, podając następujące elementy:

* **Funkcja kosztu (loss)** – miara błędu, którą sieć będzie minimalizować podczas treningu. W przypadku klasyfikacji wieloklasowej użyjemy *kategoriowej entropii krzyżowej* (**categorical crossentropy**), która porównuje rozkład prawdopodobieństw na wyjściu (softmax) z oczekiwaną etykietą (one-hot).
* **Optymalizator** – algorytm aktualizujący wagi sieci na podstawie obliczonego błędu. Wybierzemy popularny optymalizator **Adam**, który adaptacyjnie dostosowuje kroki uczenia dla każdej wagi.
* **Metryka oceny** – metryka, którą będziemy śledzić w trakcie treningu. Najbardziej naturalną metryką dla klasyfikacji jest **dokładność** (*accuracy*), czyli odsetek poprawnie sklasyfikowanych obrazów.

In [ ]:
Po kompilacji, rozpoczniemy trenowanie modelu (**fit**). Przekażemy generator danych treningowych, liczbę epok oraz generator walidacyjny do monitorowania jakości modelu na nieznanych danych podczas treningu. Trening przez co najmniej 30 epok pozwoli modelowi wielokrotnie zobaczyć wszystkie obrazy treningowe i stopniowo poprawiać swoje parametry. W trakcie uczenia Keras będzie wyświetlał postęp – stratę i dokładność na treningu oraz walidacji po każdej epoce, co pozwoli nam obserwować czy model się poprawia i czy nie zaczyna nadmiernie dopasowywać do danych treningowych (gdyby dokładność walidacji przestała rosnąć lub zaczęła spadać, mimo rosnącej dokładności treningowej, byłby to sygnał przeuczenia)

In [ ]:
# Kompilacja modelu
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Trenowanie modelu
EPOCHS = 30  # liczba epok treningu

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=valid_generator
)



W powyższym kodzie:

* `model.compile(...)` ustawia optymalizator, funkcję straty i metrykę.
* `model.fit(...)` rozpoczyna proces uczenia:
  * `train_generator` dostarcza batchy danych treningowych (obrazy i one-hot etykiety) w sposób ciągły.
  * `epochs=EPOCHS` określa, ile pełnych przebiegów przez zbiór treningowy wykonamy.
  * `validation_data=valid_generator` powoduje, że po każdej epoce model zostanie oceniony na danych walidacyjnych (to pozwala śledzić postępy na danych niewidzialnych podczas treningu).



W miarę postępu epok oczekujemy, że **loss (strata)** będzie malała, a **accuracy (dokładność)** rosła, zarówno dla danych treningowych, jak i walidacyjnych. Jeśli model ma wystarczającą pojemność, to po 30 epokach powinien osiągnąć dość wysoką dokładność na zbiorze treningowym. Ważniejsze jest jednak, jaka będzie **val\_accuracy** – to ona świadczy o zdolności uogólniania. Przy dobrze dobranej architekturze i parametrach, model powinien uzyskać dokładność walidacyjną prawdopodobnie w okolicach 80-90%. Gdyby dokładność walidacyjna była znacznie niższa od treningowej, oznaczałoby to przeuczenie i wymagało dalszych działań (np. bardziej złożonej augmentacji, mocniejszego dropout, prostszej architektury lub wczesnego zatrzymania treningu).


Po zakończeniu treningu, możemy zwizualizować przebieg procesu ucznia (np. wykres strat i dokładności) za pomocą obiektu `history`




## Ewaluacja modelu

Ostatnim krokiem jest ocena wytrenowanego modelu na zbiorze walidacyjnym (który w naszym przypadku pełni rolę zestawu testowego, gdyż model nie widział tych obrazów podczas treningu). Do oceny użyjemy metody `model.evaluate`, która obliczy wartość funkcji kosztu oraz wskazane metryki dla podanych danych.

In [ ]:
# Ewaluacja modelu na zbiorze walidacyjnym
val_loss, val_accuracy = model.evaluate(valid_generator)

print(f"Wynik na zbiorze walidacyjnym: strata = {val_loss:.4f}, dokładność = {val_accuracy*100:.2f}%")

